In [2]:
!uv pip install --system trl accelerate bitsandbytes # evaluate rouge_score
# !uv pip install --system --upgrade "protobuf<5"
!uv pip install --system --upgrade wandb

Using Python 3.12.12 environment at: /usr
Audited 3 packages in 89ms
Using Python 3.12.12 environment at: /usr
Resolved 20 packages in 79ms                                         
Audited 20 packages in 0.42ms


In [3]:
import transformers
import torch
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda:0" if torch.cuda.is_available() else "cpu"

print(f'Transformers version: {transformers.__version__}')
print(f'Device: {device}')

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Transformers version: 4.57.3
Device: cuda:0


In [3]:
DATASET_ID = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"
MODEL_ID = "LGAI-EXAONE/EXAONE-4.0-1.2B"
OUTPUT_DIR = "./exaone-4.0-customer-support-sft"

In [4]:
dataset_raw = load_dataset(DATASET_ID)['train']
print(dataset_raw)
print(dataset_raw[0])

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 26872
})
{'flags': 'B', 'instruction': 'question about cancelling order {{Order Number}}', 'category': 'ORDER', 'intent': 'cancel_order', 'response': "I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you."}


In [5]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    dtype=torch.float16, # because i used a t4 gpu
    device_map="auto",
    attn_implementation="eager", # again, t4 limitation
    )#.to("cuda")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

print(model)
print(f'Tokenizer Default Pad Token: {tokenizer.pad_token}')
print(f'Tokenizer Default Pad Side: {tokenizer.padding_side}')

Exaone4ForCausalLM(
  (model): Exaone4Model(
    (embed_tokens): Embedding(102400, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-29): 30 x Exaone4DecoderLayer(
        (self_attn): Exaone4Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Exaone4RMSNorm((64,), eps=1e-05)
          (k_norm): Exaone4RMSNorm((64,), eps=1e-05)
        )
        (mlp): Exaone4MLP(
          (gate_proj): Linear(in_features=2048, out_features=4096, bias=False)
          (up_proj): Linear(in_features=2048, out_features=4096, bias=False)
          (down_proj): Linear(in_features=4096, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (post_attention_layernorm): Exaone4RMSNorm((2048,), eps=

In [6]:
dummy_chat = [
    {"role": "user", "content": "C"},
    {"role": "assistant", "content": "B"},
]
template_output = tokenizer.apply_chat_template(dummy_chat, tokenize=False, add_generation_prompt=False, enable_thinking=False)
print(repr(template_output)) # the separator response token is "[|assistant|]\n"

'[|user|]\nC[|endofturn|]\n[|assistant|]\n<think>\n\n</think>\n\nB[|endofturn|]\n'


In [7]:
def apply_chat_style(example):
  messages = [
      {"role": "user", "content": example['instruction']},
      {"role": "assistant", "content": example['response']}
  ]

  return {
      "text": tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=False,
          enable_thinking=False)
      }

In [8]:
dataset_raw[0]

{'flags': 'B',
 'instruction': 'question about cancelling order {{Order Number}}',
 'category': 'ORDER',
 'intent': 'cancel_order',
 'response': "I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you."}

In [9]:
dataset = dataset_raw.map(apply_chat_style, remove_columns=dataset_raw.column_names)

train_test_split = dataset.train_test_split(test_size=0.1, seed=42)
train_val_split = train_test_split['train'].train_test_split(test_size=0.111, seed=42)

test_dataset = train_test_split['test']
train_dataset = train_val_split['train']
val_dataset = train_val_split['test']

In [10]:
train_dataset[0]

{'text': "[|user|]\nwhag do i need to do to cancel order {{Order Number}}[|endofturn|]\n[|assistant|]\n<think>\n\n</think>\n\nI get it that you're uncertain about the steps required to cancel order {{Order Number}}. Allow me to provide you with the guidance you need:\n\n1. Sign in to Your Account: Start by logging in to your {{Online Company Portal Info}} using your credentials.\n2. Locate Your Orders: Once you're logged in, navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.\n3. Find the Relevant Order: Look for the order with the number {{Order Number}} and click on it for more details.\n4. Initiate the Cancellation: Within the order details, you should find an option labeled '{{Online Order Interaction}}'. Please select this to begin the cancellation process.\n5. Complete Any Additional Steps: Our system might prompt you with a few questions or ask for feedback. Kindly provide the necessary information to proceed.\n\nShould you encounter any dif

In [11]:
sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,

    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=32,
    # predict_with_generate=False,

    max_steps=2500 * 3,
    eval_strategy='steps',
    eval_steps=2500 // 2,
    save_steps=100,
    save_total_limit=2,
    logging_steps=10,

    # optim="adamw_torch",
    optim="paged_adamw_8bit",
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,

    fp16=False,
    packing=False,
    report_to="wandb",
    completion_only_loss=True,
    dataset_text_field="text"
)

In [12]:
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # formatting_func=apply_chat_style,
    processing_class=tokenizer,
)

In [13]:
# import wandb

# wandb.login()

In [14]:
# model.is_parallelizable

In [15]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [16]:
trainer.train()

wandb: Currently logged in as: omid511 (omid511-org) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


KeyboardInterrupt: 